In [ ]:
import random
import secrets

import protarrow
import ptars

from ptars_protos.benchmark_pb2 import BenchmarkMessage

In [ ]:
pool = ptars.HandlerPool()
handler = pool.get_for_message(BenchmarkMessage.DESCRIPTOR)

In [ ]:
MESSAGE_COUNT = 10_000
STRING_SIZE = 10
MIN_INT = -2147483648
MAX_INT = 2147483647

In [ ]:
messages = [
    BenchmarkMessage(
        query=secrets.token_urlsafe(random.randint(0, STRING_SIZE)),
        page_number=random.randint(MIN_INT, MAX_INT),
        result_per_page=random.randint(MIN_INT, MAX_INT),
    )
    for _ in range(MESSAGE_COUNT)
]

payloads = [m.SerializeToString() for m in messages]

In [ ]:
%%timeit
protarrow.messages_to_record_batch(
    [BenchmarkMessage.FromString(p) for p in payloads],
    BenchmarkMessage,
)

In [ ]:
%%timeit
handler.list_to_record_batch(payloads)

In [ ]:
record_batch = handler.list_to_record_batch(payloads)

In [ ]:
%%timeit
handler.record_batch_to_array(record_batch)

In [ ]:
%%timeit
[
    m.SerializeToString()
    for m in protarrow.record_batch_to_messages(record_batch, BenchmarkMessage)
]